# ES → BBF Razorflow Vendor Account Migration

This notebook migrates **Razorflow Vendor** Account records from ES Salesforce to BBF Salesforce.

## Filter Criteria
- `Razorflow_Vendor__c = true` (checkbox on ES Account)

This is a separate migration from the main Account migration (02_account_migration.ipynb) which filters by BAN relationships.

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query ES Accounts where `Razorflow_Vendor__c = true` and not yet migrated
3. Transform ES Accounts for BBF schema
4. Insert Accounts to BBF Salesforce
5. Update ES Accounts with `BBF_New_Id__c` = BBF Account.Id
6. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF:** `ES_Legacy_ID__c` stores original ES Account ID

**In ES:** `BBF_New_Id__c` stores new BBF Account ID

## Safety
- `DRY_RUN = True` by default (queries and transforms but does NOT insert/update)
- `TEST_MODE = True` by default (limits to 10 Accounts)
- All inserts use bulk API for performance
- Verification queries after migration
- Excel output shows all successes and failures

In [5]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [6]:
# === CONFIGURATION ===

# ES (Source) Credentials - UAT
# ES_USERNAME = "sfdcapi@everstream.net.uat"
# ES_PASSWORD = "ZXasqw1234!@#$"
# ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
# ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'test' for sandbox

# Migration Options
DRY_RUN = False  # ⚠️ Set to False to actually insert/update records
TEST_MODE = False  # ⚠️ Set to False to process ALL Razorflow vendor accounts
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Account Owner - Set all migrated accounts to this user
OWNER_ID = "005Ea00000ZOGFZIA5"

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_razorflow_vendor_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   DRY_RUN: {DRY_RUN}")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Output: {output_file}")

if DRY_RUN:
    print("\n" + "!" * 70)
    print("DRY RUN MODE: No records will be inserted or updated!")
    print("Set DRY_RUN = False to perform actual migration.")
    print("!" * 70)
else:
    print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

📋 Configuration loaded
   DRY_RUN: False
   TEST_MODE: False
   Owner ID: 005Ea00000ZOGFZIA5
   Output: es_bbf_razorflow_vendor_migration_20260209_151016.xlsx

⚠️  Note: Bulk API automatically handles batching (200 records/batch)


In [7]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n🔌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n🔌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

🔌 Connecting to ES (source)...
✅ Connected to ES: everstream.my.salesforce.com

🔌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [8]:
# === QUERY ES RAZORFLOW VENDOR ACCOUNTS ===

print("\n" + "=" * 80)
print("QUERYING ES RAZORFLOW VENDOR ACCOUNTS")
print("=" * 80)

# Query ES Accounts where Razorflow_Vendor__c = true and not yet migrated
print("\n📌 Querying ES Accounts where Razorflow_Vendor__c = true...")

query = """
    SELECT Id, Name, Type, BillingStreet, BillingCity, BillingState, 
           BillingPostalCode, BillingCountry, Phone, Website, Industry,
           AnnualRevenue, NumberOfEmployees, Description,
           ShippingStreet, ShippingCity, ShippingState, ShippingPostalCode,
           ShippingCountry, AccountNumber, Site, TickerSymbol, Ownership,
           Rating, Sic, SicDesc
    FROM Account
    WHERE Razorflow_Vendor__c = true
      AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')
"""

# Add limit for test mode
if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query: {query[:200]}...")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_accounts_raw = result["records"]

print(f"   Found {len(es_accounts_raw)} Razorflow Vendor Accounts from ES query")

# =============================================================================
# DUPLICATE PREVENTION: Check BBF for already-migrated records
# If record exists in BBF, get its Id to update ES BBF_New_Id__c
# =============================================================================
print("\n" + "-" * 80)
print("DUPLICATE PREVENTION: Checking BBF for existing ES_Legacy_ID__c...")
print("-" * 80)

bbf_existing_query = """
SELECT Id, ES_Legacy_ID__c 
FROM Account 
WHERE ES_Legacy_ID__c != null
"""
bbf_existing_result = bbf_sf.query_all(bbf_existing_query)

# Build lookup: ES_Legacy_ID__c -> BBF Id
existing_bbf_lookup = {
    r["ES_Legacy_ID__c"]: r["Id"] for r in bbf_existing_result["records"]
}

print(f"   Found {len(existing_bbf_lookup)} Account records already in BBF")

# Separate: records to migrate vs records that need ES BBF_New_Id__c sync
es_accounts = []
es_needs_sync = []  # Records that exist in BBF but ES.BBF_New_Id__c is null

for acct in es_accounts_raw:
    if acct["Id"] in existing_bbf_lookup:
        # Already in BBF - need to sync ES.BBF_New_Id__c
        es_needs_sync.append(
            {"es_id": acct["Id"], "bbf_id": existing_bbf_lookup[acct["Id"]]}
        )
    else:
        # Not in BBF - need to migrate
        es_accounts.append(acct)

print(f"   Records to migrate (not in BBF): {len(es_accounts)}")
print(f"   Records to sync (in BBF, ES.BBF_New_Id__c missing): {len(es_needs_sync)}")

# Sync ES.BBF_New_Id__c for records that already exist in BBF
if len(es_needs_sync) > 0:
    if DRY_RUN:
        print(
            f"\n🔍 DRY RUN - Would sync {len(es_needs_sync)} ES Account BBF_New_Id__c values"
        )
    else:
        print(f"\n📌 Syncing {len(es_needs_sync)} ES Account BBF_New_Id__c values...")

        sync_updates = [
            {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
            for item in es_needs_sync
        ]

        # Use small batches to avoid CPQ trigger limits
        BATCH_SIZE = 10
        sync_success = 0
        sync_failed = 0

        try:
            for i in range(0, len(sync_updates), BATCH_SIZE):
                batch = sync_updates[i : i + BATCH_SIZE]
                results = es_sf.bulk.Account.update(batch)
                sync_success += sum(1 for r in results if r["success"])
                sync_failed += sum(1 for r in results if not r["success"])

            print(f"   ✅ Synced: {sync_success}")
            print(f"   ❌ Failed to sync: {sync_failed}")
        except Exception as e:
            print(f"   ❌ Error syncing: {e}")

print(
    f"\n✅ {len(es_accounts)} Razorflow Vendor accounts to migrate (after duplicate check)"
)

if len(es_accounts) > 0:
    sample = es_accounts[0]
    print(f"\nSample Account:")
    print(f"  ID:   {sample['Id']}")
    print(f"  Name: {sample['Name']}")
    print(f"  Type: {sample.get('Type', 'N/A')}")
    print(f"  City: {sample.get('BillingCity', 'N/A')}")
elif TEST_MODE:
    print("\n⚠️  No unmigrated Razorflow Vendor accounts found in test set")
else:
    print("\n✅ All Razorflow Vendor accounts have been migrated!")


QUERYING ES RAZORFLOW VENDOR ACCOUNTS

📌 Querying ES Accounts where Razorflow_Vendor__c = true...
Query: 
    SELECT Id, Name, Type, BillingStreet, BillingCity, BillingState, 
           BillingPostalCode, BillingCountry, Phone, Website, Industry,
           AnnualRevenue, NumberOfEmployees, Description,...

Executing query...
   Found 231 Razorflow Vendor Accounts from ES query

--------------------------------------------------------------------------------
DUPLICATE PREVENTION: Checking BBF for existing ES_Legacy_ID__c...
--------------------------------------------------------------------------------
   Found 2263 Account records already in BBF
   Records to migrate (not in BBF): 225
   Records to sync (in BBF, ES.BBF_New_Id__c missing): 6

📌 Syncing 6 ES Account BBF_New_Id__c values...
   ✅ Synced: 6
   ❌ Failed to sync: 0

✅ 225 Razorflow Vendor accounts to migrate (after duplicate check)

Sample Account:
  ID:   0010B00001oCnJIQA0
  Name: Verizon Wireless - Vendor
  Type: Vendo

In [9]:
# === TRANSFORM FOR BBF ===

print("\n" + "=" * 80)
print("TRANSFORMING ACCOUNTS FOR BBF")
print("=" * 80)

bbf_accounts = []

for es_account in es_accounts:
    bbf_account = {
        # Standard fields
        "Name": es_account.get("Name"),
        "Type": es_account.get("Type"),
        "BillingStreet": es_account.get("BillingStreet"),
        "BillingCity": es_account.get("BillingCity"),
        "BillingState": es_account.get("BillingState"),
        "BillingPostalCode": es_account.get("BillingPostalCode"),
        "BillingCountry": es_account.get("BillingCountry"),
        "ShippingStreet": es_account.get("ShippingStreet"),
        "ShippingCity": es_account.get("ShippingCity"),
        "ShippingState": es_account.get("ShippingState"),
        "ShippingPostalCode": es_account.get("ShippingPostalCode"),
        "ShippingCountry": es_account.get("ShippingCountry"),
        "Phone": es_account.get("Phone"),
        "Website": es_account.get("Website"),
        "Industry": es_account.get("Industry"),
        "AnnualRevenue": es_account.get("AnnualRevenue"),
        "NumberOfEmployees": es_account.get("NumberOfEmployees"),
        "Description": es_account.get("Description"),
        "AccountNumber": es_account.get("AccountNumber"),
        "Site": es_account.get("Site"),
        "TickerSymbol": es_account.get("TickerSymbol"),
        "Ownership": es_account.get("Ownership"),
        "Rating": es_account.get("Rating"),
        "Sic": es_account.get("Sic"),
        "SicDesc": es_account.get("SicDesc"),
        # 👤 Set account owner
        "OwnerId": OWNER_ID,
        # 🔗 Store ES Account ID for tracking
        "ES_Legacy_ID__c": es_account["Id"],
    }

    bbf_accounts.append(bbf_account)

if len(bbf_accounts) > 0:
    print(f"✅ Transformed {len(bbf_accounts)} accounts")
    print(
        f"   - Mapped {len([k for k in bbf_accounts[0].keys() if not k.startswith('_')])} fields per account"
    )
    print(f"   - Set OwnerId to {OWNER_ID}")
    print(f"   - Set ES_Legacy_ID__c for tracking")
else:
    print("⚠️  No accounts to transform")


TRANSFORMING ACCOUNTS FOR BBF
✅ Transformed 225 accounts
   - Mapped 27 fields per account
   - Set OwnerId to 005Ea00000ZOGFZIA5
   - Set ES_Legacy_ID__c for tracking


In [10]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING ACCOUNTS TO BBF")
print("=" * 80)

if DRY_RUN:
    print("\n🔍 DRY RUN - Skipping insert to BBF")
    print(f"   Would insert {len(bbf_accounts)} accounts")

    # Create simulated results for dry run
    successful_inserts = []
    failed_inserts = []

    for i, acct in enumerate(bbf_accounts):
        successful_inserts.append(
            {
                "es_id": acct["ES_Legacy_ID__c"],
                "bbf_id": f"[DRY_RUN_{i+1}]",
                "name": acct["Name"],
            }
        )

    print(f"\n   Preview of first 5 accounts that would be inserted:")
    for item in successful_inserts[:5]:
        print(f"     - {item['name']} (ES ID: {item['es_id']})")

elif len(bbf_accounts) == 0:
    print("⚠️  No accounts to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_accounts)} accounts using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Account.insert(bbf_accounts)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_accounts[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": bbf_accounts[i]["Name"],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_accounts[i]["ES_Legacy_ID__c"],
                        "name": bbf_accounts[i]["Name"],
                        "errors": result["errors"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} accounts")
        print(f"❌ Failed to insert: {len(failed_inserts)} accounts")

        if len(failed_inserts) > 0:
            print(f"\nFailed Accounts (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES ID:  {sample['es_id']}")
            print(f"  BBF ID: {sample['bbf_id']}")
            print(f"  Name:   {sample['name']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []


INSERTING ACCOUNTS TO BBF
Inserting 225 accounts using bulk API...
(Bulk API automatically batches in 200-record chunks)

✅ Successfully inserted: 225 accounts
❌ Failed to insert: 0 accounts

Sample successful insert:
  ES ID:  0010B00001oCnJIQA0
  BBF ID: 001Ea00001LyO2yIAF
  Name:   Verizon Wireless - Vendor


In [11]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if DRY_RUN:
    print("\n🔍 DRY RUN - Skipping ES update")
    print(f"   Would update {len(successful_inserts)} ES accounts with BBF_New_Id__c")

elif len(successful_inserts) == 0:
    print("⚠️  No successful inserts to update in ES")
else:
    es_updates = []
    for item in successful_inserts:
        es_updates.append({"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]})

    print(f"Updating {len(es_updates)} ES accounts with BBF IDs...")
    print("(Processing in batches of 10 to avoid CPQ trigger governor limits)\n")

    # Process in small batches to avoid CPQ trigger SOQL limit
    BATCH_SIZE = 10
    total_success = 0
    total_errors = 0

    try:
        for i in range(0, len(es_updates), BATCH_SIZE):
            batch = es_updates[i : i + BATCH_SIZE]
            batch_num = (i // BATCH_SIZE) + 1
            total_batches = (len(es_updates) + BATCH_SIZE - 1) // BATCH_SIZE

            print(
                f"  Processing batch {batch_num}/{total_batches} ({len(batch)} records)...",
                end=" ",
            )

            results = es_sf.bulk.Account.update(batch)

            success_count = sum(1 for r in results if r["success"])
            error_count = sum(1 for r in results if not r["success"])

            total_success += success_count
            total_errors += error_count

            if error_count > 0:
                print(f"⚠️  {success_count} OK, {error_count} failed")
                for j, result in enumerate(results):
                    if not result["success"]:
                        print(f"    Error on {batch[j]['Id']}: {result['errors']}")
            else:
                print(f"✅ {success_count} OK")

        print(f"\n✅ Successfully updated: {total_success} ES accounts")
        if total_errors > 0:
            print(f"❌ Failed to update: {total_errors} ES accounts")

        print("\n🔄 Bidirectional tracking complete:")
        print("   BBF → ES_Legacy_ID__c (original ES ID)")
        print("   ES → BBF_New_Id__c (new BBF ID)")

    except Exception as e:
        print(f"⚠️  Could not update ES accounts: {e}")
        print("(Migration can continue - ES tracking is optional)")


UPDATING ES WITH BBF IDS
Updating 225 ES accounts with BBF IDs...
(Processing in batches of 10 to avoid CPQ trigger governor limits)

  Processing batch 1/23 (10 records)... ✅ 10 OK
  Processing batch 2/23 (10 records)... ✅ 10 OK
  Processing batch 3/23 (10 records)... ✅ 10 OK
  Processing batch 4/23 (10 records)... ✅ 10 OK
  Processing batch 5/23 (10 records)... ✅ 10 OK
  Processing batch 6/23 (10 records)... ✅ 10 OK
  Processing batch 7/23 (10 records)... ✅ 10 OK
  Processing batch 8/23 (10 records)... ✅ 10 OK
  Processing batch 9/23 (10 records)... ✅ 10 OK
  Processing batch 10/23 (10 records)... ✅ 10 OK
  Processing batch 11/23 (10 records)... ✅ 10 OK
  Processing batch 12/23 (10 records)... ✅ 10 OK
  Processing batch 13/23 (10 records)... ✅ 10 OK
  Processing batch 14/23 (10 records)... ✅ 10 OK
  Processing batch 15/23 (10 records)... ✅ 10 OK
  Processing batch 16/23 (10 records)... ✅ 10 OK
  Processing batch 17/23 (10 records)... ✅ 10 OK
  Processing batch 18/23 (10 records)... 

In [12]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

# Combine all results
all_results = []

for item in successful_inserts:
    all_results.append(
        {
            "ES_Account_Id": item["es_id"],
            "BBF_Account_Id": item["bbf_id"],
            "Account_Name": item["name"],
            "Status": "DRY_RUN" if DRY_RUN else "SUCCESS",
            "Error": "",
        }
    )

for item in failed_inserts:
    all_results.append(
        {
            "ES_Account_Id": item["es_id"],
            "BBF_Account_Id": "",
            "Account_Name": item["name"],
            "Status": "FAILED",
            "Error": str(item["errors"]),
        }
    )

# Identify name collisions
name_collisions = []
for item in failed_inserts:
    error_str = str(item["errors"])
    if "DUPLICATE_VALUE" in error_str and "Name" in error_str:
        name_collisions.append(
            {
                "ES_Account_Id": item["es_id"],
                "Account_Name": item["name"],
                "Error_Details": error_str,
            }
        )

print(f"Identified {len(name_collisions)} name collision errors")

# Create Excel workbook
wb = Workbook()
wb.remove(wb.active)

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

status_colors = {"SUCCESS": "00C851", "FAILED": "FF4444", "DRY_RUN": "FFA500"}

# --- SHEET 1: Migration Results ---
ws1 = wb.create_sheet("Migration Results")
headers1 = ["ES Account ID", "BBF Account ID", "Account Name", "Status", "Error"]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_Account_Id"],
            r["BBF_Account_Id"],
            r["Account_Name"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Razorflow Vendor Account Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])

# Determine run type
if DRY_RUN:
    run_type = "DRY RUN (no changes made)"
elif TEST_MODE:
    run_type = "TEST MODE"
else:
    run_type = "FULL MIGRATION"

ws2.append(["Run Type:", run_type])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A7"].font = Font(bold=True)
ws2["B7"].font = Font(bold=True)
ws2.append(["Total Accounts Processed", len(all_results)])
ws2.append(
    ["Successful Inserts" if not DRY_RUN else "Would Insert", len(successful_inserts)]
)
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Name Collisions", len(name_collisions)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "N/A"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Account ID", "BBF Account ID", "Account Name"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["name"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Account ID", "Account Name", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            str(item["errors"]),
        ]
    )

for col in ws4.columns:
    max_length = max(len(str(cell.value)) for cell in col) if list(col) else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} accounts, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(
    f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} {'planned' if DRY_RUN else 'successful'} mappings)"
)
print(f"   ❌ Sheet 4: Failed Inserts ({len(failed_inserts)} errors)")


CREATING EXCEL OUTPUT
Identified 0 name collision errors

✅ Excel output saved to: es_bbf_razorflow_vendor_migration_20260209_151016.xlsx
   📊 Sheet 1: Migration Results (225 accounts, color-coded)
   📈 Sheet 2: Summary (metrics and stats)
   🔗 Sheet 3: ID Mapping (225 successful mappings)
   ❌ Sheet 4: Failed Inserts (0 errors)


In [13]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE" if not DRY_RUN else "DRY RUN COMPLETE")
print("=" * 80)
print(f"ES Razorflow Vendor Accounts queried: {len(es_accounts)}")
print(
    f"BBF Accounts {'inserted' if not DRY_RUN else 'would be inserted'}: {len(successful_inserts)}"
)
if len(es_accounts) > 0:
    print(f"Success rate: {len(successful_inserts)/len(es_accounts)*100:.1f}%")
else:
    print("N/A - no accounts to migrate")
print(f"\nExcel output: {output_file}")

if DRY_RUN:
    print("\n" + "!" * 70)
    print("🔍 DRY RUN COMPLETE - No records were inserted or updated!")
    print("   Review the Excel output to verify the planned migration.")
    print("   To perform actual migration:")
    print("   1. Set DRY_RUN = False in Cell 2")
    print("   2. Optionally set TEST_MODE = False to migrate all 231 accounts")
    print("   3. Re-run all cells")
    print("!" * 70)
elif TEST_MODE:
    print(
        "\n🔄 TEST MODE complete. Only migrated up to " + str(TEST_LIMIT) + " accounts."
    )
    print(
        "   To migrate ALL Razorflow Vendor accounts, set TEST_MODE = False in Cell 2 and re-run."
    )
else:
    print("\n✅ FULL MIGRATION complete!")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} accounts failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")


MIGRATION COMPLETE
ES Razorflow Vendor Accounts queried: 225
BBF Accounts inserted: 225
Success rate: 100.0%

Excel output: es_bbf_razorflow_vendor_migration_20260209_151016.xlsx

✅ FULL MIGRATION complete!


---
## Rollback Commands (Salesforce Apex)

### Delete Migrated Razorflow Vendor Accounts from BBF
```apex
// Find accounts migrated from ES (have ES_Legacy_ID__c)
List<Account> a = [SELECT Id, Name, ES_Legacy_ID__c FROM Account WHERE ES_Legacy_ID__c != null AND Owner.Name = 'Everstream Legacy'];
System.debug(a);
delete a;
```

### Clear BBF_New_Id__c from ES Razorflow Vendor Accounts
```apex
List<Account> accountIds = [SELECT Id, BBF_New_Id__c FROM Account WHERE Razorflow_Vendor__c = true AND BBF_New_Id__c != NULL];
System.debug(accountIds);
for (Account a : accountIds){
    a.BBF_New_Id__c = NULL;
}
update accountIds;
```